In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install langchain
!pip install torch
!pip install accelerate
!pip install sentence-transformers
!pip install streamlit
!pip install streamlit-chat
!pip install faiss-cpu
!pip install tiktoken
!pip install huggingface-hub
!pip install pypdf
!pip install llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  U

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyPDFLoader
import os
import tempfile

In [ ]:
def initialize_session_state():
    session_state = {
        "history": [],
        "generated": ["Hello! Ask me anything about 🤖"],
        "past": ["Hey! 👋"]
    }
    return session_state


In [ ]:
def conversation_chat(query, chain, session_state):
    result = chain({"question": query, "chat_history": session_state["history"]})
    session_state["history"].append((query, result["answer"]))
    return result["answer"]


In [ ]:
def collect_user_questions():
    questions = []
    print("Enter your questions one by one and type 'done' when finished:")
    while True:
        user_input = input("Enter your question: ")
        if user_input.lower() == 'done':
            break
        questions.append(user_input)
    return questions

In [ ]:
def display_chat_history(chain, session_state):
    questions = collect_user_questions()
    for question in questions:
        answer = conversation_chat(question, chain, session_state)
        print(f"Question: {question}")
        print(f"Answer: {answer}\n")


In [ ]:
def create_conversational_chain(vector_store):
    print('Creating conversational chain...')
    print("Started creating LLM...")

    llm = LlamaCpp(
        streaming=True,
        model_path="/content/drive/MyDrive/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
        temperature=0.75,
        top_p=1,
        verbose=True,
        n_ctx=4096,
    )

    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
        memory=memory,
    )

    print("Completed creating LLM!")

    return chain

In [ ]:
def main():
    session_state=initialize_session_state()
    print("ChatBot using Mistral-7B-Instruct LLM :books:")

    uploaded_files = ["/content/Chainpoll A high efficacy method for LLM hallucination detection - 2310.18344.pdf"]

    if uploaded_files:
        text = []
        for file_path in uploaded_files:
            with open(file_path, "rb") as f:
                file_contents = f.read()
            file_extension = os.path.splitext(file_path)[1]
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_file.write(file_contents)
                temp_file_path = temp_file.name
                print("loading: ", file_path)
            loader = None
            if file_extension == ".pdf":
                loader = PyPDFLoader(temp_file_path)

            if loader:
                text.extend(loader.load())
                os.remove(temp_file_path)

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=10000, chunk_overlap=20
        )
        text_chunks = text_splitter.split_documents(text)

        print("chunks:\n", text_chunks)

        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={"device": "cpu"},
        )

        print("embeddings:\n", embeddings)

        vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

        chain = create_conversational_chain(vector_store)

        display_chat_history(chain,session_state)


if __name__ == "__main__":
    main()

ChatBot using Mistral-7B-Instruct LLM :books:
loading:  /content/Chainpoll A high efficacy method for LLM hallucination detection - 2310.18344.pdf
chunks:
 [Document(page_content='ChainPoll : A H IGHEFFICACY METHOD FOR LLM\nHALLUCINATION DETECTION\nRobert Friel\nGalileo Technologies Inc.Atindriyo Sanyal\nGalileo Technologies Inc.\nOctober 31, 2023\nABSTRACT\nLarge language models (LLMs) have witnessed significant advancements in generating coherent, intelligent,\nand contextually relevant responses. However, the presence of hallucinations – inaccurate or unmotivated\nclaims – remains a persistent challenge, motivating the development of automated metrics for the detection of\nhallucinations in LLM outputs.\nWe make two contributions: ChainPoll , a novel hallucination detection methodology that substantially out-\nperforms existing alternatives, and RealHall , a carefully curated suite of benchmark datasets for evaluating\nhallucination detection metrics proposed in recent literature.\n

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Question: What is the significance of ChainPoll achieving superior performance across all four benchmarks in RealHall?
Answer:  The significance of ChainPoll achieving superior performance across all four benchmarks in RealHall is that it demonstrates the effectiveness of the method for detecting both open-domain and closed-domain hallucinations. It also highlights the importance of carefully engineering prompts for hallucination detection, as well as the value of Boolean judgments over numeric scores.




llama_print_timings:        load time =     850.28 ms
llama_print_timings:      sample time =      14.06 ms /    26 runs   (    0.54 ms per token,  1848.56 tokens per second)
llama_print_timings: prompt eval time =   20482.22 ms /   168 tokens (  121.92 ms per token,     8.20 tokens per second)
llama_print_timings:        eval time =    4901.41 ms /    25 runs   (  196.06 ms per token,     5.10 tokens per second)
llama_print_timings:       total time =   25525.85 ms /   193 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     850.28 ms
llama_print_timings:      sample time =      77.08 ms /   138 runs   (    0.56 ms per token,  1790.46 tokens per second)
llama_print_timings: prompt eval time =  231663.20 ms /  1686 tokens (  137.40 ms per token,     7.28 tokens per second)
llama_print_timings:        eval time =   27041.36 ms /   137 runs   (  197.38 ms per token,     5.07 tokens per second)
llama_print_timings:       total time =  259733.30 ms /  1823 

Question: How does RealHall address the limitations of existing datasets used in prior work on hallucination detection?
Answer:   Limitations of existing datasets in prior work on hallucination detection include lack of diversity, challenge, and realism. RealHall addresses these limitations by carefully selecting four datasets that meet criteria for Challenge, Realism, and Task Diversity. For example, RealHall Closed evaluates how well a metric can detect closed-domain hallucinations in Retrieval Augmented Generation (RAG) use cases by using COVID-QA with retrieval and DROP. RealHall Open evaluates how well a metric can detect open-domain hallucinations in realistic settings by using the Open Assistant prompts and TriviaQA datasets.




llama_print_timings:        load time =     850.28 ms
llama_print_timings:      sample time =      14.84 ms /    26 runs   (    0.57 ms per token,  1752.26 tokens per second)
llama_print_timings: prompt eval time =   43756.84 ms /   334 tokens (  131.01 ms per token,     7.63 tokens per second)
llama_print_timings:        eval time =    4943.24 ms /    25 runs   (  197.73 ms per token,     5.06 tokens per second)
llama_print_timings:       total time =   48893.82 ms /   359 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     850.28 ms
llama_print_timings:      sample time =      64.00 ms /   113 runs   (    0.57 ms per token,  1765.71 tokens per second)
llama_print_timings: prompt eval time =  232252.50 ms /  1686 tokens (  137.75 ms per token,     7.26 tokens per second)
llama_print_timings:        eval time =   22547.03 ms /   112 runs   (  201.31 ms per token,     4.97 tokens per second)
llama_print_timings:       total time =  255760.80 ms /  1798 

Question: What are the key contributions of ChainPoll in the field of hallucination detection for LLMs?
Answer:   The main limitations of existing datasets used in prior work on hallucination detection are that they do not meet the criteria for challenge, realism, and task diversity. RealHall addresses these limitations by carefully selecting four datasets that meet these criteria and dividing them into two groups of two: RealHall Closed and RealHall Open. The former evaluates how well a metric can detect closed-domain hallucinations while the latter tests open-domain hallucinations. This allows for more comprehensive evaluation of metrics in real-world use cases.

